## Caso COVID-19 Fase 1
Julián Correa, Guillermo Cortés, Cristian Sarmiento

## Creditos de la información
La data utilizada en el ejercicio es tomada del **repositorio de datos COVID-19 del Centro de Ciencias de Datos e Ingeniria de Sistemas (CSSE) de la Universidad Johns Hopkins [CSSEGISandData COVID-19 GitHub Repository](https://github.com/CSSEGISandData/COVID-19)** 

### Preprocesarmiento de información

In [1]:
## import libraries and dependencies
import pandas as pd

## read downloaded csv file
covid_cases = pd.read_csv('time_series_covid19_confirmed_global.csv')
covid_cases.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/23,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,209322,209340,209358,209362,209369,209390,209406,209436,209451,209451
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,334391,334408,334408,334427,334427,334427,334427,334427,334443,334457
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,271441,271448,271463,271469,271469,271477,271477,271490,271494,271496
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,47866,47875,47875,47875,47875,47875,47875,47875,47890,47890
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,105255,105277,105277,105277,105277,105277,105277,105277,105288,105288


Al realizar la inspección nos encontramos con que la tabla esta en formato horizontan donde las fechas son columnas en el data frame lo que hace complicado el procesamiento de la información. Procedemos hacer el comando **melt** de pandas para convertir **date** en variable y **value** como otra variable que contiene el numero de casos.

In [2]:
## melt dataframe to get dates
id_vars = ['Province/State', 'Country/Region', 'Lat', 'Long']
df = covid_cases.melt(id_vars=id_vars, var_name='date')
df.sample(n=5)

,Province/State,Country/Region,Lat,Long,date,value
177970,NaN,Slovakia,48.6690,19.6990,9/28/21,800730
324321,Gansu,China,35.7518,104.2861,2/17/23,1742
173599,Sint Maarten,Netherlands,18.0425,-63.0548,9/13/21,3999
234333,NaN,Sudan,12.8628,30.2176,4/11/22,62034
303597,NaN,Iceland,64.9631,-19.0208,12/7/22,207101


## Revisión de tipos de variables

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330327 entries, 0 to 330326
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Province/State  104013 non-null  object 
 1   Country/Region  330327 non-null  object 
 2   Lat             328041 non-null  float64
 3   Long            328041 non-null  float64
 4   date            330327 non-null  object 
 5   value           330327 non-null  int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 15.1+ MB


Encontramos que la variable **date** no corresponde al tipo de valor de fecha, procedemos hacer el ajuste convirtiendo **object** a tipo **datetime64**

In [4]:
## Transform date column to datetime format
df['date'] = pd.to_datetime(df['date'], format='%m/%d/%y')
df.sample(n=5)

,Province/State,Country/Region,Lat,Long,date,value
313218,NaN,Serbia,44.0165,21.0059,2023-01-09,2454074
48171,"Bonaire, Sint Eustatius and Saba",Netherlands,12.1784,-68.2385,2020-07-06,7
13682,NaN,Croatia,45.1000,15.2000,2020-03-09,12
191019,NaN,Uruguay,-32.5228,-55.7658,2021-11-12,396175
98321,Chongqing,China,30.0572,107.8740,2020-12-27,590


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330327 entries, 0 to 330326
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Province/State  104013 non-null  object        
 1   Country/Region  330327 non-null  object        
 2   Lat             328041 non-null  float64       
 3   Long            328041 non-null  float64       
 4   date            330327 non-null  datetime64[ns]
 5   value           330327 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 15.1+ MB


In [6]:
# Se elimina la columna Province/State
df.drop('Province/State', axis=1, inplace=True)
# Los valores que estaban desagregados por la variable Province/State, se hace un groupby Pais, Fecha y se suman
df = df.groupby(['Country/Region', 'date'])['value'].sum().reset_index()

## 1. ¿En cuál mes se presentó el mayor número de contagios?

In [7]:
# Se crea una copia para procesar el data frame
df_sorted = df.copy()
# Se ordena el arreglo
df_sorted.sort_values(by=['Country/Region', 'date'], inplace=True)
# Se hace el calcula de los casos diarios restanda el valor del arreglo con el valor previo.
df_sorted['new_cases'] = df_sorted.groupby(["Country/Region"])['value'].diff().fillna(df_sorted['value'])
# se crea una columna nueva para facilitar los conteos con el valore del año-mes
df_sorted['month_year'] = df_sorted['date'].dt.to_period('M')
df_sorted

,Country/Region,date,value,new_cases,month_year
0,Afghanistan,2020-01-22,0,0.0,2020-01
1,Afghanistan,2020-01-23,0,0.0,2020-01
2,Afghanistan,2020-01-24,0,0.0,2020-01
3,Afghanistan,2020-01-25,0,0.0,2020-01
4,Afghanistan,2020-01-26,0,0.0,2020-01
...,...,...,...,...,...
229738,Zimbabwe,2023-03-05,264127,0.0,2023-03
229739,Zimbabwe,2023-03-06,264127,0.0,2023-03
229740,Zimbabwe,2023-03-07,264127,0.0,2023-03
229741,Zimbabwe,2023-03-08,264276,149.0,2023-03


### Validación si hay valores negativos

In [8]:
# Se identifican si hay valores negativos
negative_df = df_sorted[df_sorted['new_cases'] < 0].dropna()

Al encontrar valores negativos vamos asumir que fueron errores de digitación y los vamos a imputar con 0.

In [9]:
# Se eliminan valores negativos y se vuelve a verificar
df_sorted['new_cases'] = df_sorted['new_cases'].apply(lambda x: max(x, 0))
negative_df = df_sorted[df_sorted['new_cases'] < 0].dropna()
negative_df

,Country/Region,date,value,new_cases,month_year


In [10]:
# Se hace una agregacion y se suman todos los valores por mes
df_agg_month = df_sorted.groupby(['month_year'], as_index=False)['new_cases'].sum()
df_agg_month.sample(n=10)

,month_year,new_cases
21,2021-10,13065754.0
9,2020-10,12122070.0
28,2022-05,16262773.0
13,2021-02,11253787.0
5,2020-06,4316300.0
32,2022-09,14734569.0
10,2020-11,17312616.0
2,2020-03,783399.0
7,2020-08,7942806.0
27,2022-04,25229296.0


In [11]:
# Se obtiene el mes con mas contagios
cond_max_cases = df_agg_month['new_cases'] == df_agg_month['new_cases'].max()
df_agg_month[cond_max_cases]

,month_year,new_cases
24,2022-01,90511973.0


In [12]:
max_month = df_agg_month[cond_max_cases]['month_year'].iloc[0]
total_cases = df_agg_month[cond_max_cases]['new_cases'].iloc[0]
print(f'El mes y el año con mas casos es {max_month} con un toltal de {total_cases:,}')

El mes y el año con mas casos es 2022-01 con un toltal de 90,511,973.0


## 2. ¿En ese mismo mes, cuál fue el país que reportó más contagios?

In [13]:
# Se filtra solo el mes con más contagios del paso anrterior
cond = (df_sorted['month_year'] == max_month)
df_country = df_sorted[cond].groupby(['Country/Region'], as_index=False)['new_cases'].sum()
df_country.sort_values(by='new_cases').sample(n=10)

,Country/Region,new_cases
34,Central African Republic,1756.0
64,Gabon,5127.0
156,Senegal,9944.0
188,Ukraine,407980.0
3,Andorra,12218.0
56,Equatorial Guinea,2092.0
162,Slovenia,246773.0
147,Russia,1349961.0
104,Liechtenstein,2797.0
16,Belarus,43843.0


In [14]:
# Se obtiene el pais con mas contagios en el mes del paso anterior
max_country = df_country[df_country['new_cases'] == df_country['new_cases'].max()]
print(f"El pais con mas casos es {max_country.iloc[0]['Country/Region']} con un toltal de {max_country.iloc[0]['new_cases']:,}")

El pais con mas casos es US con un toltal de 20,336,435.0


## 3. ¿Cuál es el país con el menor número de casos reportados hasta la fecha?

In [25]:
# Se obtiene la última fecha del reporte
max_fecha = df['date'].max()
# Se filtra por la ultima fecha
df_max_fecha = df[df['date'] == max_fecha]
# Se obtiene el index de la fila con el menor número de casos
idx_min_value = df_max_fecha['value'].idxmin()
# Se obtiene la fila con el menor número de casos
row_min_value = df_max_fecha.loc[idx_min_value]
print("El pais con el valor mínimo de casos reportados:")
print(row_min_value)

El pais con el valor mínimo de casos reportados:
Country/Region           Korea, North
date              2023-03-09 00:00:00
value                               1
Name: 107441, dtype: object


## Importancia de la limpieza y preparación de datos con pandas en el contexto de la ciencia a de datos

1. **Mejora la calidad de los datos:** Al hacer este proceso le damos consistencia, precisión e integridad a los datos. Consitencia al eliminar los datos nulos o que no concuerdan con la realidad. Precisión al eliminar valores atípicos o con errores. Por ultimo integridad al garantizar que toda la data este completa.
2. **Facilita el análisis posterior:** Al tener los datos limpios nos permiten ejecutar los modelos predictivos de manera mas precisa y con visualizaciones claras.
3. **Ahorro de tiempo:** Evita reprocesos al momento de correr los modelos.
4. **Estandarización:** Normaliza los datos de forma coherente, facilitando la integración con otros conjuntos de datos.

## ¿Por qué es importante ejecutar un proceso de limpieza y preparacion antes de almacenar los datos en un data lake?

1. Al eliminar datos que no agregan valor al análisis se ahorra espacio,  ahorros económicos y de performance ya que los datos innecesarios relentizan las consultas.
2. El tener los datos limpios se ahorra tiempo al momento de correr los modelos, ya que se tendríá ejecutar un proceso de limpieza sobre toda la data antes de ejecutar los modelos
3. Un data lake recibe información de muchas fuentes, con la normalización de los datos se garantiza que no importa la fuente est va a ser homogenea.
4. 

***
___